In [5]:
import pandas as pd
import pywhatkit as wpp
from datetime import datetime 

#SetOptions para ver los dataframes
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',False)

Lectura del Excel

In [6]:
path = 'Data/'
df = pd.read_excel(path + "Román - M4 Seguimiento.xlsx", sheet_name = "Extras")
df02 = pd.read_excel(path + "BASE BLENDED NL S1M04 2024.xlsx", sheet_name = 'ACTIVOS')
#df = pd.read_csv("M4Seguimiento.csv")
#df02 = pd.read_csv("Informacion.csv",skiprows=1, usecols= lambda column: column != "Contacto efectivo")

FileNotFoundError: [Errno 2] No such file or directory: 'Data/BASE BLENDED NL S1M04 2024.xlsx'

Limpieza de Columnas

In [ ]:
df = df[["Matricula","Nombre","Materia","Clave Carrera", "Tipo Calificacion", "Correo", "Centro Enlace"]]
df.shape

Filtran solo de Cadereyta y CC Cadereyta

In [ ]:
df = df[df["Centro Enlace"].isin(["CADEREYTA", "CC CADEREYTA"])]

Se agrupan en una nueva columna las materias del alumno

In [ ]:
df_agrupado = df.groupby("Nombre")["Materia"].agg(lambda x: ", ".join(x)).reset_index().rename(columns = {"Materia": "Materias"})
df_final = pd.merge(df, df_agrupado, on = "Nombre", how = "left")     
df_final

Se intercambian de lugar las columnas para mayor comodidad. 
Se Elimina la fila de Materias al final ya que pasa al frente.
Filtro de duplicados. Renombra la columna Materia por Materia.

In [ ]:
#Cambio de columna y eliminar la extra
columna_extra = df_final["Materias"]
df_final.drop(columns = ["Materias"], inplace=True)
df_final["Materia"] = columna_extra

#Filtra dúplicados
df_final = df_final.drop_duplicates( subset = "Nombre")
#Renombra
df_final.rename(columns = {"Materia" : "Materias"}, inplace= True)
df_reprobados = df_final
df_reprobados


#Con la información en el DF02 buscamos conservar solo los alumnos ya que hay muchos más registros sin alumnos.

(Ya no, ahora el df es el BASE BLENDED)

In [ ]:
#ultimo_nombre = 1 + df02["NOMBRE"].last_valid_index()
#df02_final = pd.read_csv("Informacion.csv",skiprows=1, usecols= lambda column: column != "Contacto efectivo", nrows= ultimo_nombre)
#df02_final = df02_final.drop(df02_final.index[0])
#df02_final.shape

In [ ]:
df_final[df_final['Matricula'] == 'BNL081349']

Se eliminan todas las columnas después de la columna CELULAR. 
Se elimina la columna TUTOR ya que está vacía.
Cambia el formato de las columnas TELÉFONO y CELULAR

In [ ]:
#Se eliminan columnas hasta la columna Teléfono 2.
columna_limite = "Teléfono2"
columnas_conservadas = df02.columns[:df02.columns.get_loc(columna_limite) + 1]
df02 = df02[columnas_conservadas] 

#Se elimina USUARIO BB,PASSWORD PORTAL ALUMNO, PROYECTO, MODELO EDUCATIVO, SERVICIO, Modalidad, Monto, Correo 2
df02.drop(columns = ["USUARIO BB","PASSWORD PORTAL \nALUMNO", 'PROYECTO','MODELO EDUCATIVO', 'Fecha', 'SERVICIO', 'Fecha de inicio','Modalidad', 'Monto', 'Correo 2'], inplace=True)

#Renombrar el nombre de la columna NOMBRE
df02.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE"}, inplace= True)
df02.rename(columns = {'MATRICULA' : "Matricula"}, inplace= True)
#Cambia el formato de las columnas TELÉFONO y CELULAR
#df02_final["TELÉFONO"] = df02_final["TELÉFONO"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
#df02_final["CELULAR"] = df02_final["CELULAR"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
df02

Solo CADEREYTA

In [ ]:
contacto = df02[df02['CE CANALIZADO'] == 'CE CADEREYTA']
contacto#[contacto['Matricula'] == 'BNL102027']

Lista de los NOMBRES y MATRICULAS de los Reprobados

In [ ]:
reprobados = df_reprobados["Nombre"].unique()
matricula_reprobados = df_reprobados['Matricula'].unique()
matricula_reprobados

Reprobados que están en el Módulo 4

Tabla comprendida en la cual solo está la información de los reprobados que están en Módulo 4

In [ ]:
reprobados_mod4 = contacto[(contacto['Matricula'].isin(matricula_reprobados)) & (contacto['M4'] == 4)]
reprobados_mod4 = pd.merge(reprobados_mod4, df_reprobados[['Matricula', 'Materias', 'Tipo Calificacion']], on = 'Matricula')
reprobados_mod4

Mensaje predeterminado para alumnos

In [ ]:
for index, row in reprobados_mod4.iterrows():
    alumno = row['NOMBRE']
    matricula_reprobados = row['Matricula']
    tipoExamen = row['Tipo Calificacion']
    materias = row['Materias']
    telefono = row['Teléfono']
    telefono = '+52 '+str(telefono)
    mensaje = 'Hola, que tal ' + alumno + ' ' + matricula_reprobados + ', soy Román Rodríguez tu Tutor Presencial del CNCI, me comunico contigo debido a que veo que dejaste en ' + tipoExamen + 'la(s) siguiente(s) materia(s): ' + materias +'. ¿Me gustaría saber si puedo apoyarte con el proceso de tus materias en extraordinario?.'
    hora = int(datetime.now().strftime('%H'))
    min = int(datetime.now().strftime('%M'))+1
    #wpp.sendwhatmsg(telefono, mensaje, hora, min )
    print(telefono,'\n' , mensaje,'\n','HORA:', hora, 'MIN', min)

# Consultas

In [ ]:
df_reprobados[df_reprobados['Matricula']== 'BNL088083']

In [ ]:
contacto[contacto['Matricula']== 'BNL088083']